In [1]:
# Import necessary libraries
import pandas as pd
import tensorflow_decision_forests as tfdf
import tensorflow as tf



2024-02-28 18:24:39.913595: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 18:24:39.913669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-28 18:24:39.953836: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-28 18:24:40.048627: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 18:24:41.070011: W tensorflow/compiler/tf2

In [2]:

# Load your dataset
data_path = 'combined_csv.csv'  # Update this path
data = pd.read_csv(data_path)



In [3]:
# Split the data into features and target
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]



In [4]:
# Convert to TensorFlow dataset and split into train, validation, and test sets
def df_to_tf_dataset(features, target):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), target))
    return dataset



In [5]:
# Splitting the data (adjust split sizes as needed)
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
test_size = len(data) - train_size - val_size

full_dataset = df_to_tf_dataset(X, y)
full_dataset = full_dataset.shuffle(buffer_size=len(data))

# Adjusted dataset preparation without shuffle operation
train_dataset = full_dataset.take(train_size).batch(16)
val_dataset = full_dataset.skip(train_size).take(val_size).batch(16) # Ensure validation dataset is correctly defined
test_dataset = full_dataset.skip(train_size + val_size).batch(16)


2024-02-28 18:24:49.529254: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 18:24:49.702885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 18:24:49.702982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 18:24:49.706899: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 18:24:49.706957: I external/local_xla/xla/stream_executor

In [6]:
# Train a Random Forest model with dataset check disabled (if needed)
model = tfdf.keras.RandomForestModel(check_dataset=False)
model.fit(train_dataset)

Use /tmp/tmpqn69h2w5 as temporary training directory


Reading training dataset...
Training dataset read in 0:00:03.925426. Found 2189 examples.
Training model...
Model trained in 0:00:00.206987
Compiling model...


[INFO 24-02-28 18:24:59.1138 +0530 kernel.cc:1233] Loading model from path /tmp/tmpqn69h2w5/model/ with prefix 44b33a13417b4c18
[INFO 24-02-28 18:24:59.1657 +0530 decision_forest.cc:660] Model loaded with 300 root(s), 39900 node(s), and 8 input feature(s).
[INFO 24-02-28 18:24:59.1658 +0530 abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-02-28 18:24:59.1658 +0530 kernel.cc:1061] Use fast generic engine


Model compiled.


In [7]:
# Evaluate the model on the test set
evaluation = model.evaluate(test_dataset)



30/30 [==============================] - 0s 4ms/step - loss: 0.0000e+00


In [8]:
# Plotting feature importances
model.summary()



Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (8):
	person_center_x
	person_center_y
	person_intersection_percentage
	person_velocity
	vehicle_center_x
	vehicle_center_y
	vehicle_intersection_percentage
	vehicle_velocity

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.                "vehicle_center_x"  0.383788 ################
    2.                "vehicle_center_y"  0.306749 ###########
    3.                 "person_center_x"  0.235241 #######
    4.  "person_intersection_percentage"  0.230301 ######
    5.                 "person_center_y"  0.183364 ###
    6. "vehicle_intersection_percentage"  0.164471 ##
    7.         

In [9]:
# Built-in TensorFlow Decision Forests Plotting
# Note: This requires TensorFlow Decision Forests version >= 0.2.0

# Plotting model structure and feature importances
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)
tfdf.model_plotter.plot_feature_importances(model)


AttributeError: module 'tensorflow_decision_forests.component.model_plotter.model_plotter' has no attribute 'plot_feature_importances'

In [10]:

# Note: The `plot_model_in_colab` function is designed to work in Google Colab and might not render directly in Jupyter.
# For Jupyter, you can use `model.make_inspector().export_to_tensorboard('/path/to/logdir')` and visualize with TensorBoard.

# Advanced model insights
inspector = model.make_inspector()
print("Model type:", inspector.model_type())
print("Number of trees:", inspector.num_trees())
print("Objective:", inspector.objective())
print("Input features:", inspector.features())
print("Evaluation:", inspector.evaluation())



Model type: RANDOM_FOREST
Number of trees: 300
Objective: Classification(label=__LABEL, class=None, num_classes=2)
Input features: ["person_center_x" (1; #1), "person_center_y" (1; #2), "person_intersection_percentage" (1; #3), "person_velocity" (1; #4), "vehicle_center_x" (1; #5), "vehicle_center_y" (1; #6), "vehicle_intersection_percentage" (1; #7), "vehicle_velocity" (1; #8)]
Evaluation: Evaluation(num_examples=2189, accuracy=0.9748743718592965, loss=0.09167369996725512, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)


In [11]:
# Optional: Save your model for later use
model.save('your_model_path')

INFO:tensorflow:Assets written to: your_model_path/assets


INFO:tensorflow:Assets written to: your_model_path/assets
